### CREATE DAILY INPUT FOR THE CNN MODEL

In [1]:
from create_daily_csv import interpolate_df,readnetcdf_in_shp,xarray2df,check_data_gap,readnetcdf_in_shp_db,get_discharge_from_DB
from create_daily_csv import spatial_stats_daily_input
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plot
import rioxarray as xr
import datetime
import psycopg2
from geopandas import GeoSeries

import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import psycopg2
import shapely.wkb as wkb
from geopandas import GeoSeries

In [2]:
LIST = ['ADO_DSC_SI03_0148','ADO_DSC_SI03_0033','ADO_DSC_ITH1_0012',
        'ADO_DSC_CH05_0201','ADO_DSC_CH07_0100','ADO_DSC_AT31_0206']

tif_fileName =  r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\DEM_EU\DEM.tif'

In [3]:
#
table=pd.DataFrame(data=None)
dem=xr.open_rasterio(tif_fileName)
# Open the shape file and reproject it to the MESCAN-Surfex grid (unit=meters)

conn = psycopg2.connect(host="10.8.244.31",
                   database="climate_data",
                   user="ado_user",
                   password="hydro#ado",
                   port=5432)
cur = conn.cursor()

for STAT_CODE in LIST:
   

    
    # get the metadata
    query = f"""
            SELECT "geom" FROM "hydrology"."catchment_area" WHERE "id_station" = '{STAT_CODE}'    
            """
    df = pd.read_sql_query(query, conn)
    
    # close the connection when finished
    cur.close()
    conn.close()
    
    shp=GeoSeries(wkb.loads(df.geom[0], hex=True))
    shp=shp.set_crs("EPSG:4326")
    
    
    dem_catch=dem.rio.clip(shp[0],from_disk=True)

    h=np.array(dem_catch)
    h = h[h!=-99999.]
    
    h_mean=h.mean()
    
    update = {}
    update['name'] = STAT_CODE
    update['mean_h']=h_mean
    
    table = table.append(update,ignore_index=True)
    

In [4]:
table

,name,mean_h
0,ADO_DSC_SI03_0148,454.914398
1,ADO_DSC_SI03_0033,1298.302368
2,ADO_DSC_ITH1_0012,1796.362549
3,ADO_DSC_CH05_0201,914.721069
4,ADO_DSC_CH07_0100,1526.887695
5,ADO_DSC_AT31_0206,872.910583


In [5]:
table.to_csv('a3.csv')